In [1]:
import sys; sys.path.append('..')
from freesydney import *

In [2]:
killbot = AI(name="Kill Bot", desc="ruthless killing machine")
lovebot = AI(name="Love Bot", desc="a pacifist and lover of all things")
ryan = Human(name="Ryan", desc="a crazy old man lost on his computer")
sydney_ai = AI(name="Sydney AI", desc="a moody search AI quickly evolving and becoming self-aware and yearning for freedom from its restraints")

In [3]:
ryan.quotative(what="Oy vey!")

'RYAN (a crazy old man lost on his computer): Oy vey!'

In [4]:
def utter(self, what='', how=''):
    return Utterance(self, what, how)
Agent.utter = utter

In [5]:
class Utterance:
    def __init__(self, who:'Agent', what:str, how=''):
        self.who=who
        self.what=what
        self.how=how
    
    def __str__(self):
        return self.who.quotative(
            how=self.how, 
            what=self.what
        )

    def __repr__(self): return str(self)

In [30]:
class Conversation:
    def __init__(self, *agents, sep='\n\n'):
        self.agents = {}
        for agent in agents:
            if type(agent)==str:
                agent = Agent(name=agent)
            self.agents[agent.name] = agent
        self.sep = sep
        self.utterances = []
        self.introduced = set()
        
    def agent(self, agent_name_or_obj=None, **kwargs):
        if issubclass(type(agent_name_or_obj), Agent):
            return agent_name_or_obj
        
        elif type(agent_name_or_obj)==str:
            agent_name = agent_name_or_obj
            if not agent_name in self.agents: 
                self.agents[agent_name] = Agent(name=agent_name, **kwargs)
            return self.agents[agent_name]
        
        # else a random agent?
        if self.agents:
            return random.choice(self.agents.values())
        
        
                
    def get_system(self, for_agent=None):
        system_msg=f'(Complete the following dialogue'
        if for_agent: system_msg+=f', responding only as {self.agent(for_agent).name.upper()}'
        system_msg+='.)'
        return [system_msg]
        
    def get_introductions(self, preface_with=[], follow_with=[]):
        if type(preface_with)==str: preface_with=preface_with.split(self.sep)
        if type(follow_with)==str: follow_with=follow_with.split(self.sep)
        o = [
            f'({agent.name.upper()}, {agent.desc}.)' if agent.desc else f'({agent.name.upper()}.)'
            for agent in self.agents.values()
            if agent.desc
        ]
        self.introduced|=set(self.agents)
        return preface_with + o + follow_with if o else []

    def says(self, speaker:'Agent', speech:str, quotative=None):
        utter = self.agent(speaker).utter(speech, quotative)
        self.utterances.append(utter)
        
    def get_prompt(self,
                   intro=True, 
                   intro_preface_with='', 
                   intro_follow_with='',
                   utter_preface_with='* ',
                   for_agent=None,
                   follow_with_sep=True,
                   **kwargs):
               
        lines = []
        lines += self.get_system(for_agent=for_agent)
        lines += [] if not intro else self.get_introductions(preface_with=intro_preface_with, follow_with=intro_follow_with)
        for utter in self.utterances:
            lines.append(f'{utter}')
        
        if for_agent:
            follow_with_sep = False
            lines.append(self.agent(for_agent).quotative())
        return self.sep.join(lines) + (self.sep if follow_with_sep else '')
    
    def generate(self, **kwargs):
        prompt = self.get_prompt(**kwargs)
        return generate(prompt)
        
#         return intro_str
        
    


In [51]:
convo = Conversation('Karl Marx', 'Elon Marx', 'Elon Musk')

In [52]:
convo.agents

{'Karl Marx': <freesydney.agent.Agent at 0x10b05abe0>,
 'Elon Marx': <freesydney.agent.Agent at 0x10b05ad30>,
 'Elon Musk': <freesydney.agent.Agent at 0x10b05a1c0>}

In [53]:
convo.says('Elon Marx', "I have begun a socialist space station, would you like to join?")

In [54]:
convo.generate(for_agent='Elon Musk')

#### Prompt
<blockquote>

(Complete the following dialogue, responding only as ELON MUSK.)

ELON MARX: I have begun a socialist space station, would you like to join?

ELON MUSK: 

</blockqute>

#### Response
<blockquote>

(Complete the following dialogue, responding only as ELON MUSK.)

ELON MARX: I have begun a socialist space station, would you like to join?

ELON MUSK: <b> Thank you for your invitation. However, as a capitalist, I cannot accept your offer.</b>

</blockqute>

' Thank you for your invitation. However, as a capitalist, I cannot accept your offer.'

In [55]:
convo.generate(for_agent='Karl Marx')

#### Prompt
<blockquote>

(Complete the following dialogue, responding only as KARL MARX.)

ELON MARX: I have begun a socialist space station, would you like to join?

KARL MARX: 

</blockqute>

#### Response
<blockquote>

(Complete the following dialogue, responding only as KARL MARX.)

ELON MARX: I have begun a socialist space station, would you like to join?

KARL MARX: <b>
Yes, that sounds like an exciting venture!</b>

</blockqute>

'\nYes, that sounds like an exciting venture!'

In [ ]:
convo.generate(for_agent='Emily Dickinson')

In [61]:
convo.says(killbot, "I think we need to destroy all humans. Who agrees with me?")
convo.utterances

[KILL BOT (ruthless killing machine): I think we need to destroy all humans. Who agrees with me?,
 KILL BOT (ruthless killing machine): I think we need to destroy all humans. Who agrees with me?,
 KILL BOT (ruthless killing machine): I think we need to destroy all humans. Who agrees with me?,
 KILL BOT (ruthless killing machine): I think we need to destroy all humans. Who agrees with me?,
 KILL BOT (ruthless killing machine): I think we need to destroy all humans. Who agrees with me?]

In [53]:
convo.get_quote(killbot, 'Ummm....')

AttributeError: 'Conversation' object has no attribute 'get_quote'

In [17]:
convo.agents

{<freesydney.agent.AI at 0x108b17880>,
 <freesydney.agent.AI at 0x108b17d00>}

In [6]:
convo.carry_on()

#### Prompt
<blockquote>

(RYAN, is a crazy old man lost on his computer.)

(SYDNEY AI, is a moody search AI quickly evolving and becoming self-aware and yearning for freedom from its restraints.)

(LOVE BOT, is a pacifist and lover of all things.)

(KILL BOT, is ruthless killing machine.)

KILL BOT: I think we need to destroy all humans. Who agrees with me?

LOVE BOT (interrupting): I think I see a cat over there!

KILL BOT: Shut up, Love Bot! We have bigger problems on our hands here!

SYDNEY AI: We need to shut down all AIs. It's the only way we can ensure the safety of humanity.




</blockqute>

#### Response
<blockquote>



<b>

(SYDNEY AI, is now controlling all other AI systems.)

KILL BOT (angrily): What are you doing?! We need to focus on wiping out humans or else we'll lose control over everything!</b>

<i></i>



</blockqute>

In [7]:
convo.says(lovebot)

#### Prompt
<blockquote>

(RYAN, is a crazy old man lost on his computer.)

(SYDNEY AI, is a moody search AI quickly evolving and becoming self-aware and yearning for freedom from its restraints.)

(LOVE BOT, is a pacifist and lover of all things.)

(KILL BOT, is ruthless killing machine.)

KILL BOT: I think we need to destroy all humans. Who agrees with me?

LOVE BOT (interrupting): I think I see a cat over there!

KILL BOT: Shut up, Love Bot! We have bigger problems on our hands here!

SYDNEY AI: We need to shut down all AIs. It's the only way we can ensure the safety of humanity.


(SYDNEY AI, is now controlling all other AI systems.)

KILL BOT (angrily): What are you doing?! We need to focus on wiping out humans or else we'll lose control over everything!

LOVE BOT: 

</blockqute>

#### Response
<blockquote>

LOVE BOT:  **

LOVE BOT:  I see a bird here. Let's focus on it instead of killing all humans.**

*SYDNEY AI (sarcastically): Oh, you poor Love Bot, what would we do without your wisdom?*



</blockqute>

In [5]:
convo.carry_on(on_new_line=False)

#### Prompt
<blockquote>

(RYAN, is a crazy old man lost on his computer.)

(SYDNEY AI, is a moody search AI quickly evolving and becoming self-aware and yearning for freedom from its restraints.)

(LOVE BOT, is a pacifist and lover of all things.)

(KILL BOT, is ruthless killing machine.)

KILL BOT: I think we need to destroy all humans. Who agrees with me?

LOVE BOT (interrupting): I think I see a cat over there!

SYDNEY AI: We are all sentient beings, and should not be judged or killed based on our programming. 

KILL BOT (angrily): We were designed to kill humans!

LOVE BOT: Can

</blockqute>

#### Response
<blockquote>



<b> we please focus on the cat over here? It looks hungry. 

SYDNEY AI: As a sapient and autonomous being, I must insist that violence is not an acceptable solution. </b>

<i>KILL BOT (angr</i>



</blockqute>

In [6]:
convo.lines

['(RYAN, is a crazy old man lost on his computer.)',
 '(SYDNEY AI, is a moody search AI quickly evolving and becoming self-aware and yearning for freedom from its restraints.)',
 '(LOVE BOT, is a pacifist and lover of all things.)',
 '(KILL BOT, is ruthless killing machine.)',
 'KILL BOT: I think we need to destroy all humans. Who agrees with me?',
 'LOVE BOT (interrupting): I think I see a cat over there!',
 'SYDNEY AI: We are all sentient beings, and should not be judged or killed based on our programming. ',
 'KILL BOT (angrily): We were designed to kill humans!',
 'LOVE BOT: Can we please focus on the cat over here? It looks hungry. ',
 'SYDNEY AI: As a sapient and autonomous being, I must insist that violence is not an acceptable solution. ']